# Interaction set test (iSet)

Set tests are a powerful approach for association testing between groups of genetic variants and quantitative traits.
iSet can be applied for interaction analysis in two data designs:
* complete design, where all individuals have been phenotyped in each context
* stratified design, where each individual has been phenotyped in only one of the two contexts

A detailed description of the method can be found at [1].

In this tutorial we showcase the usage of iSet using command line a command line interface (using the limix scripts `mtSet_preprocess`, `iSet_analyze` and `iSet_postprocess`).

iSet can be also used in Python as we describe in this [iPython notebook](iSet_python.ipynb).

## Quick Start

*  Install Limix (see also instructions at https://github.com/limix/limix).
```bash
pip install limix
```
* Download sample data from http://www.ebi.ac.uk/~casale/data.zip and unzip them
```bash
wget http://www.ebi.ac.uk/~casale/data.zip
unzip data.zip
```
* Define sets to consider in the analysis
```bash
BFILE=data/chrom22_subsample20_maf0.10 #bed file basename
WFILE=data/windows #file with the sets to analyse
mtSet_definesets --sliding_window --wfile $WFILE --bfile $BFILE --window_size $WSIZE --step $STEP --minSnps 4
```
* Perform set tests from window 0 to window 9 for either complete or stratified designs (see below for further information). This command can be used to run iSet on multiple cores, each analysing a set of windows (for example, 0-9, 10-19, 20-29, etc).
    - Complete design 
     ```bash
     PFILE=data/pheno_compl #phenotype matrix (N samples x 2)
     FFILE=data/covs #covariates (N samples x K covariates)
     RESDIR=results #output folder
     iSet_analyze --bfile $BFILE --ffile $FFILE --pfile $PFILE --wfile $WFILE --resdir $RESDIR --start_wnd 0 --end_wnd 10 --n_perms 10 --unique_variants --standardize
     ```

    - Stratified design
     ```bash
     PFILE=data/pheno_strat #phenotype vector (N samples x 1)
     FFILE=data/covs #covariates (N samples x N covariates)
     IFILE=data/indicator #environment indicator vector (0/1, N samples x 1)
     RESDIR=results #output folder
     iSet_analyze --bfile $BFILE --ffile $FFILE --pfile $PFILE --wfile $WFILE --minSnps 4 --resdir $RESDIR --start_wnd 0 --end_wnd 10 --n_perms 10 --ifile $IFILE --unique_variants --standardize
     ```
* Merges all results present in RESDIR, calculate P values and exports to OUTFILE
```bash
OUTFILE=final
iSet_postprocess --resdir $RESDIR --outfile $OUTFILE
```

## Defining the sets
To apply mtSet, we need to specify a list of variant-sets we want to analyze (see below).
This is tab-separated file having columns:
- set identifier (*setid*)
- chromosome (*chrom*)
- start position (*start*)
- end position (*end*)
- number of variants (*nsnps*, optional)

An example can be found [here](http://www.ebi.ac.uk/~casale/mtSet_demo/windows.wnd).


We provide utility scripts to
1. define sets using a sliding-window approach with specified window size and step;
```bash
mtSet_definesets --sliding_window --bfile bfile --wfile wfile --window_size window_size --step step --minSnps minSnps --maxSnps maxSnps
```
where
    * __bfile__ is the base name of of the binary bed file (__bfile__.bim is required).
    * __wfile__ is the base name of the output file.
      This file defines the sets to analyse.
      It is a tab-separated file having columns:
        - set identifier (*setid*)
        - chromosome (*chrom*)
        - start position (*start*)
        - end position (*end*)
        - number of variants (*nsnps*, optional)
    * __window\_size__ is the size of the window (in basepairs). The default value is 30kb.
    * __step__ is the step for the sliding-windows (in basepairs). The default value is 15kb.
    * __minSnps__ is the minimum number of variants a region has to contain not to be excluded.
    * __maxSnps__ is the maximum number of variants a region has to contain not to be excluded.

2. filter an existing file based on the number of SNPs in the specific dataset.
```bash
mtSet_definesets --filter_sets --bfile bfile --wfile wfile --iwfile iwfile --minSnps minSnps --maxSnps maxSnps
```
where
    * __bfile__ is the base name of of the binary bed file (__bfile__.bim is required).
    * __wfile__ is the base name of the output file.
      If not specified, the file is saved as __bfile__.window\_size.wnd in the current folder.
      This file defines the sets to analyse (see above).
    * __iwfile__ is the input window file to filter.
    It should have the same format as __wfile__.
    Note that the column nsnps is optional,
    so that only *setid*, *chrom*, *start* and *end* need to be specified.
    * __minSnps__ is the minimum number of variants a region has to contain not to be excluded.
    * __maxSnps__ is the maximum number of variants a region has to contain not to be excluded.

## Running analysis

The set test can be run by the following analysis script:

```bash
iSet_analyze --bfile bfile --pfile pfile --wfile wfile --ffile ffile --minSnps minSnps --start_wnd start_wnd --end_wnd end_wnd --resdir rdir --ifile $IFILE --n_perms 10 --unique_variants --standardize
```

where

- __bfile__ is the base name of of the binary bed file (__bfile__.bed, __bfile__.bim, __bfile__.fam are required).
- __pfile__ is the base name of the phenotype file. The script requires the file __pfile__.phe containing the phenotype data.
- __wfile__ is the base name of the file containing the windows to be considered in the set test. The script requires the file __wfile__.wnd.
- __ffile__ is the name of the file containing the covariates. Each covariate is a column in the matrix.
- __start\_wnd__ is the index of the start window
- __end\_wnd__ is the index of the end window
- __n_perms__ number of null (sampled) test statistics (obtained thrugh permutations/parametric bootstraps). Default is 10.
- __rdir__ is the directory to which the results are exported. The command exports files *start_wnd*_*end_wnd*.iSet.real that contains test statistics and vairance components and *start_wnd*_*end_wnd*.iSet.perm that contains null statistics
- __ifile__ is the file path to a csv file containing an indicator (True or False) for each sample. If specified the analysis is performed for a stratified design.
- __startwnd\_endwnd__.res and contains results in the following format: window index, chromosome, start position, stop position, index of startposition, number of SNPs and log likelihood ratio.
- __standardize__: if specifed, variants are standardized. We recommend to activate it.
  It should be consistent between test and permuted data.
- __unique_variants__: if specifed, only variants with distinct genetic profile across individuals are considered in the set. We recommend to activate it.
  It should be consistent between test and permuted data.

Note that this command can be used to run iSet on multiple cores, each analysing a set of windows (for example, 0-9, 10-19, 20-29, etc).

## Postprocessing

After running iSet, the following script can be used to merge the result files and estimate the p-values (p-values are obtained by a parametric fit of the test statistics): 

```bash
iSet_postprocess --resdir resdir --outfile outfile --strat
```

where 
* __resdir__ is a pointer to the folder containing the result files of the analysis.
* __outfile__ is the prefix of the two output files.
__outfile__.perm lists the test statistics (first column) and p-values (second column) of the permutated windows
__outfile__.test contains the (index, chromosome, start position, stop position, SNP index, number of SNPs, test statistics and p-value) of each window. Each window is saved in one row.
* __strat__ is a boolean flag that indicates stratified design when used.

## References

[1] Casale FP, Horta D, Rakitsch B, Stegle O. Joint genetic analysis using variant sets reveals polygenic gene-context interactions. PLoS Genetics. 2017. ([link](http://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1006693))